<center><h3>Universidade Federal de Pelotas - Cursos de Ciência e Engenharia de Computação</h3></center> 
<center><h4>Disciplina: Cálculo Numérico Computacional Profa. Larissa A. de Freitas</h4> </center> 

<center><h2><strong>Relatório 4 – Diferenciação e Integração Numérica<strong></h2> </center> 

<center><h3> Aluno: Vinícius Hallmann</h3></center>

### **Imports** ###

In [152]:
import math
import numpy as np
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt

### **Métodos** ###

#### **Trapézio** ####

In [153]:
trap = lambda h, f0, f1: h*( f0 + f1 )/2

def multi_trap( h, n, f ):
    sum = f[0]
    for i in range( 1, n ):
        sum += 2 * f[i]
    sum += f[n]
    return h * sum / 2

#### **Simpson** ####

In [154]:
simp13 = lambda h, f0, f1, f2: 2*h*(f0 + 4*f1 + f2)/6

simp38 = lambda h, f0, f1, f2, f3: 3*h*(f0 + 3*(f1+f2) + f3)/8

def multi_simp_13( h, n, f ):
    sum = f[0]
    for i in range( 1, n-1, 2 ):
        sum += 4*f[i] + 2*f[i+1]
    sum += 4*f[n-1] + f[n]
    return h*sum/3

def simp_int( a, b, n, f ):
    h = ( b - a )/n
    sum = 0
    if n == 1:
        sum = trap(h,f[n-1],f[n])
    else:
        m = n
        odd = n / 2 - int(n / 2)
        if odd > 0 and n > 1:
            sum += simp38(h,f[n-3],f[n-2],f[n-1],f[n])
            m = n - 3
    
    if m > 1:
        sum += multi_simp_13(h,m,f)
    
    return sum

#### **Euler** ####

In [155]:
def euler( func, interval: np.array, y0: int, m: int, h: float = 0.1) -> np.array:
    
    h = ( interval[1] - interval[0] ) / m
    x = interval[0]
    y = y0
    vet_x = [x]
    vet_y = [y]
    fxy = func( x,y )
    for i in range( 1, m + 1 ):
        x = interval[0] + i * h
        y = y + h * fxy
        fxy = func( x,y )
        vet_x.append( x )
        vet_y.append( y )
        
    return vet_x, vet_y

### **Questão 1** ###

O   trabalho   realizado   por   uma   força   F(x)   cujo   ângulo   entre   a   direção   do movimento e a força é dado por θ(x), pode ser obtido pela seguinte fórmula: 

$$
W = \int_{x_0}^{x_n} F(x) \cos(\theta(x)) \, dx
$$

Em que x₀ e xₙ são as posições inicial e final, respectivamente. Calcule a melhor aproximação ao trabalho realizado, W, ao puxar um bloco da posição 0 até à posição 30 sabendo que a força aplicada e o ângulo usado são dados na tabela seguinte.

| X    |  0  |  2.5  |  5  |  15  |  20  |  25  |  30  |
|------|-----|-------|-----|------|------|------|------|
| f(x) |  0  |   7   |  9  |  14  | 10.5 |  12  |   5  |
| θ(x) | 0.5 |  0.9  | 1.4 |  0.9 |  1.3 | 1.48 |  1.5 |

**Observação: Verificam-se 3 grupos de espaçamentos distintos.**

In [156]:
x_values      = np.array( [  0, 2.5,   5,  15,   20,  25,   30], dtype=float )
f_values      = np.array( [  0,   7,   9,  14, 10.5,  12,    5], dtype=float )
theta_degrees = np.array( [0.5, 0.9, 1.4, 0.9,  1.3, 1.48, 1.5], dtype=float )

theta_radians = []
for degrees in theta_degrees:
    theta_radians.append( math.radians( degrees ) )
    
f_cos_values = []
for i in range( len( f_values ) ):
    f_cos_values.append( f_values[i] * math.cos( theta_degrees[i] ) )

#### **Aplicar o Método de 1/3 de Simpson Repetido no Grupo 1, o qual possui 2 subintervalos, h = 2,5.** #### 


In [157]:
W = multi_simp_13( 2.5, len( f_cos_values[:3]) - 1, f_cos_values[:3] )
print( W )

15.778986164733977


#### **Aplicar   o   Método   do   Trapézio   Repetido   no   Grupo   2,   o   qual   possui   1 subintervalo, h = 10.** #### 


In [158]:
W = multi_trap( 10, len( f_cos_values[2:4] ) - 1, f_cos_values[2:4] )
print( W )

51.16121920945736


#### **Aplicar o Método do 3/8 Simpson Repetido no Grupo 3, o qual possui 3 subintervalos, h = 5.** #### 

In [159]:
W = simp_int( 15, 30, len( f_cos_values[3:] ) - 1, f_cos_values[3:] )
print( W )

38.89990714972025


Somando todos os valores temos: $$ Wtotal = 15.778986164733977 + 51.16121920945736 + 38.89990714972025 = 105.840112524 $$

### **Questão 2** ###

Para os problemas de valor inicial, calcule   **y(0,5**)   para   o   problema   A   e   **y(0,2)**   para   o   problema   B   usando   o Método de Euler.  Considere h = **0,1** e h = **0,05**.

#### **Problema A** #####
$$
\begin{array}{ll}  
    y' = x^2 + y^2 \\
    y(0) = 0 \\
\end{array}
$$



In [160]:
f = lambda x, y: x**2 + y**2
interval = np.array( [0, 0.5], dtype=float )
y0 = 0

xa1, ya1 = euler( f, interval, y0, 5 )
xa2, ya2 = euler( f, interval, y0, 10 )

ra1 = pd.DataFrame( {'x': np.round( xa1, 3 ), 'y': np.round( ya1, 5 )} )
ra2 = pd.DataFrame( {'x': np.round( xa2, 3 ), 'y': np.round( ya2, 5 )} )
ra = pd.concat( [ra1,ra2], axis=1, keys=['0.1','0.05'] )

Considerando h = **0.1** o valor de y( 0, 5) é de 0.03002. Já com h = **0.05** chegamos a um valor de y = 0.03568


#### **Problema B** #####
$$
\begin{array}{ll}  
    y’ = x + y \\
    y(0) = 1\\
\end{array}
$$

In [161]:
f = lambda x, y: x + y
interval = np.array( [0, 0.2], dtype=float )
y0 = 1

xb1, yb1 = euler( f, interval, y0, 2 )
xb2, yb2 = euler( f, interval, y0, 4 )

rb1 = pd.DataFrame( {'x': np.round( xb1, 3 ), 'y': np.round( yb1, 5 )} )
rb2 = pd.DataFrame( {'x': np.round( xb2, 3 ), 'y': np.round( yb2, 5 )} )
rb = pd.concat( [rb1, rb2], axis=1, keys=['0.1','0.05'] )


Considerando h = **0.1** ao se aproximar de 0.2 o valor de y é aproximadamente 1.22. Já considerando h = **0.05** chegamos a um valor de y aproximado em 1.231

#### **Compare os resultados obtidos na letra do problema A e B com h = 0,1 e h = 0,05.** ####

##### **Tabela Resultados A** #####

In [162]:
print( tabulate ( ra, headers='keys', tablefmt='pretty', showindex=True ) )

+----+--------------+--------------+---------------+---------------+
|    | ('0.1', 'x') | ('0.1', 'y') | ('0.05', 'x') | ('0.05', 'y') |
+----+--------------+--------------+---------------+---------------+
| 0  |     0.0      |     0.0      |      0.0      |      0.0      |
| 1  |     0.1      |     0.0      |     0.05      |      0.0      |
| 2  |     0.2      |    0.001     |      0.1      |    0.00013    |
| 3  |     0.3      |    0.005     |     0.15      |    0.00063    |
| 4  |     0.4      |    0.014     |      0.2      |    0.00175    |
| 5  |     0.5      |   0.03002    |     0.25      |    0.00375    |
| 6  |     nan      |     nan      |      0.3      |    0.00688    |
| 7  |     nan      |     nan      |     0.35      |    0.01138    |
| 8  |     nan      |     nan      |      0.4      |    0.01751    |
| 9  |     nan      |     nan      |     0.45      |    0.02553    |
| 10 |     nan      |     nan      |      0.5      |    0.03568    |
+----+--------------+-------------

##### **Tabela Resultados B** #####

In [163]:
print( tabulate ( rb, headers='keys', tablefmt='pretty', showindex=True ) )

+---+--------------+--------------+---------------+---------------+
|   | ('0.1', 'x') | ('0.1', 'y') | ('0.05', 'x') | ('0.05', 'y') |
+---+--------------+--------------+---------------+---------------+
| 0 |     0.0      |     1.0      |      0.0      |      1.0      |
| 1 |     0.1      |     1.1      |     0.05      |     1.05      |
| 2 |     0.2      |     1.22     |      0.1      |     1.105     |
| 3 |     nan      |     nan      |     0.15      |    1.16525    |
| 4 |     nan      |     nan      |      0.2      |    1.23101    |
+---+--------------+--------------+---------------+---------------+


#### **Precisão dos Resultados** ####



Passo maior ( ℎ = 0.1 ):

 - Problema A: O valor de y( 0.5 ) é 0.03002.  ( 5 execuções )

 - Problema B: O valor de y( 0.2 ) é 1.22.     ( 3 execuções )


Passo menor ( ℎ = 0.05 ):

 - Problema A: O valor de y(0.5) é 0.03568.    ( 10 execuções )

 - Problema B: O valor de y(0.2) é 1.231.      (  5 execuções )


Usar um passo menor geralmente resulta em uma maior precisão, pois o método de Euler aproxima melhor a solução verdadeira ao tomar passos mais pequenos. No entanto, isso vem ao custo de realizar mais cálculos, o que pode aumentar o tempo de computação.